# Setup

In [ ]:
# !pip install wandb -qqq
# import wandb
# wandb.login()

     |████████████████████████████████| 1.6 MB 5.5 MB/s 
     |████████████████████████████████| 97 kB 6.1 MB/s 
     |████████████████████████████████| 133 kB 43.2 MB/s 
     |████████████████████████████████| 170 kB 52.5 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


KeyboardInterrupt: ignored

In [ ]:
# for run in range(1):
  # 1️⃣ Start a new run to track this script
  # wandb.init(
  #     # Set entity to specify your username or team name
  #     # ex: entity="aralevski",
  #     # Set the project where this run will be logged
  #     project="SciBERT", 
  #     # Track hyperparameters and run metadata
  #     config={
  #     "learning_rate": 0.02,
  #     "architecture": "CNN",
  #     "dataset": "golden_abbr.csv",})

In [1]:
!git clone https://github.com/nasa-petal/PeTaL-labeller.git

Cloning into 'PeTaL-labeller'...
remote: Enumerating objects: 16194, done.
remote: Counting objects: 100% (1660/1660), done.
remote: Compressing objects: 100% (997/997), done.
remote: Total 16194 (delta 896), reused 1363 (delta 635), pack-reused 14534
Receiving objects: 100% (16194/16194), 347.16 MiB | 20.62 MiB/s, done.
Resolving deltas: 100% (5270/5270), done.


In [2]:
%cd PeTaL-labeller/auto-labeler/scibert_new_tokens/
!ls

/content/PeTaL-labeller/auto-labeler/scibert_new_tokens
auto_labeler_prototype_scibert.py		     test.py
sagemaker					     training_loss.png
SINGLELABEL_Colleen_and_Alex_training_data_4.19.csv  train.py


In [3]:
!pip install transformers
import nltk
nltk.download("punkt")

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 22.6 MB/s 
     |████████████████████████████████| 59 kB 6.4 MB/s 
     |████████████████████████████████| 596 kB 41.1 MB/s 
     |████████████████████████████████| 3.3 MB 34.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Dataset Construction

Construct `golden.csv` to feed into SciBERT.

Find a copy of `golden.json`. Here we do this by downloading the PeTaL dataset using `setup.py` in MATCH, but you can also find it in David's `90-golden-json-copy` branch in the `data-collection-and-prep` repository on GitHub.

In [4]:
%cd ../MATCH
!python setup.py

/content/PeTaL-labeller/auto-labeler/MATCH
[2021-11-09 17:23:18,035:urllib3.connectionpool] Starting new HTTPS connection (1): drive.google.com:443
[2021-11-09 17:23:18,284:urllib3.connectionpool] https://drive.google.com:443 "GET /uc?id=1MbmyMzUkd-ke4Tnl-scTuspPmnWcRb6x HTTP/1.1" 200 None
[2021-11-09 17:23:18,289:urllib3.connectionpool] Starting new HTTPS connection (1): docs.google.com:443
[2021-11-09 17:23:18,486:urllib3.connectionpool] https://docs.google.com:443 "GET /uc?export=download&confirm=dake&id=1MbmyMzUkd-ke4Tnl-scTuspPmnWcRb6x HTTP/1.1" 200 None
[2021-11-09 17:23:18,661:urllib3.connectionpool] https://docs.google.com:443 "GET /uc?export=download&confirm=Avvm&id=1MbmyMzUkd-ke4Tnl-scTuspPmnWcRb6x HTTP/1.1" 302 None
[2021-11-09 17:23:18,664:urllib3.connectionpool] Starting new HTTPS connection (1): doc-00-20-docs.googleusercontent.com:443
[2021-11-09 17:23:18,693:urllib3.connectionpool] https://doc-00-20-docs.googleusercontent.com:443 "GET /docs/securesc/51vncomb5mgbilc86ucb

Open `golden.json`, convert it to csv so it follows the same format as `SINGLELABEL_Colleen_and_Alex_training_data_4.19.csv`, and write out to `golden.csv`

In [5]:
import json

with open('src/MATCH/PeTaL/golden.json') as fin:
    golden = json.loads(fin.read())

In [6]:
def convert_json_to_csv(js):
    title = ' '.join(js['title'])
    abstract = ' '.join(js['abstract'])
    single_labels = js['isBiomimicry']
    return ',"' + title + '","' + abstract + '",,,,' + single_labels + ","

In [7]:
with open('../scibert_new_tokens/golden.csv', 'w') as fout:
    fout.write(',title,abstract,labels,doi,url,single_labels,labels_string\n')
    for js in golden:
        fout.write(convert_json_to_csv(js) + '\n')

Just some analysis we did so we could get the column names

In [8]:
!head -n 2 ../scibert_new_tokens/golden.csv

,title,abstract,labels,doi,url,single_labels,labels_string
,"building home foam tungara frog foam nest architecture three phase construction process","frogs build foam nests floating water face problems over dispersion secretions eggs dangerously exposed air interface nest construction behaviour tungara engystomops pustulosus features may circumvent pairs periodic bursts production egg deposition three discrete phases discernible first characterized bubble raft without approximately linear increase duration mixing events time phase reduce initial precursor materials until critical concentration achieved main building marked start intervals nearly constant final change exponential fashion joining colonial nesting abbreviate presumably exploiting pioneer pair thereby reducing energy material expenditure predators finally deposited centre continuously 1 cm deep free cortex protectively encloses hatched larvae stranded",,,,Y,


In [9]:
%cd ../scibert_new_tokens

/content/PeTaL-labeller/auto-labeler/scibert_new_tokens


The full `golden.csv` was too big so we used the first 3000 papers instead

In [10]:
!head -n 3001 golden.csv > golden_abbr.csv

In [11]:
!wc golden_abbr.csv

   3001  344028 2895522 golden_abbr.csv


# Model Running

Okay, first, here's what `auto_labeler_prototype_scibert.py` contains for the function `setup_dataset()` (I think the rest is unchanged):

```
def setup_dataset():
    """Setup the Train and Validation datasets

    Returns:
        (tuple): tuple containing:

            - **train_dataset** (torch.utils.data.TensorDataset): pitch distribution
            - **val_dataset** (torch.utils.data.TensorDataset): pitch to chord distribution
    """
    ########################################
    # WE CHANGED filename IN OUR MEETING
    ########################################
    filename = 'golden_abbr.csv' # 'golden.csv' # 'SINGLELABEL_Colleen_and_Alex_training_data_4.19.csv'
    if (not os.path.exists(filename)):
        url = 'https://drive.google.com/file/d/1eOLNOl6ZMz4UxQ7qbSI-bJSSNkmNZjr9/view?usp=sharing'
        gdown.download(url, filename, quiet=False)
        md5 = 'fa837a88f0c40c513d975104edf3da17'
        gdown.cached_download(url, filename, md5=md5, postprocess=gdown.extractall)

    df = pd.read_csv(filename)
    df = df[['title', 'abstract', 'labels', 'doi', 'url', 'single_labels', 'labels_string']]
    df = df[df['single_labels'].notnull()]

    labels = []
    docs = []
    labels_test = []
    docs_test = []
    ########################################
    # WE ALSO CHANGED labels_dict AND I THINK THAT'S IT?
    ########################################
    labels_dict = ['Y', 'N'] # ["'Protect from harm'", "'Process resources'", "'Sense send or process information'", "'Maintain structural integrity'", "'Move'", "'Attach'", "'Maintain ecological community'", "'Chemically modify or Change energy state'", "'Change size or color'", "'Physically assemble/disassemble'"]

    single_labels = df["single_labels"].tolist()
    abstract = df["abstract"].tolist()
    title = df["title"].tolist()
    for i in range(len(title)):
        if i < len(title) - 40:
            docs.append(abstract[i])
            labels.append(labels_dict.index(single_labels[i]))
        else:
            docs_test.append(abstract[i])
            labels_test.append(labels_dict.index(single_labels[i]))

    print("Number of training labels: {:}".format(len(labels)))
    print("Number of training docs: {:}".format(len(docs)))
    print("Number of test labels: {:}".format(len(labels_test)))
    print("Number of test docs: {:}".format(len(docs_test)))
    
    regular_list = [word_tokenize(a) for a in abstract]
    tokens_flat = [item for sublist in regular_list for item in sublist]

    tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
    tokenizer.add_tokens(tokens_flat)

    print('SciBERT tokenizer loaded')

    # original abstract
    print(' Original: ', docs[5])
    # abstract split into tokens
    print('Tokenized: ', tokenizer.tokenize(docs[5]))
    # abstract as mapped to ids
    print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(docs[5])))
   
    # Finishing tokenizing all docs and map tokens to thier word IDs
    input_ids = []
    attention_masks = []
    actual_labels_test = []

    for d in docs:
        encoded_dict = tokenizer.encode_plus(
                            d,                      
                            truncation=True,
                            add_special_tokens = True, 
                            max_length = 256,           
                            pad_to_max_length = True,
                            return_attention_mask = True,   
                            return_tensors = 'pt',     
                    )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    print('Original: ', docs[5])
    print('Token IDs:', input_ids[5])
    print('Reverse:', tokenizer.convert_ids_to_tokens(input_ids[5]))

    # Split up training & testing/validation
    dataset = TensorDataset(input_ids, attention_masks, labels)

    # Number of docs to include per set
    train_size = int(0.9 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    return train_dataset, val_dataset, tokenizer
```

Now you run it! This part took an hour and 53 seconds to run for me on Colab

Also note that you may want to change the number of test labels/docs (it's hard-coded in `auto_labeler_prototype_scibert.py`.

In [19]:
!python binary_labeler_scibert.py --epochs=10
# wandb.log({"acc":acc, "loss":loss})

  # Mark the run as finished
# wandb.finish()

python3: can't open file 'binary_labeler_scibert.py': [Errno 2] No such file or directory
